In [35]:
import pdfplumber
import pymongo
import pandas as pd
from pymongo import MongoClient
import pandas as pd


In [36]:
def extract_dimensions(pdf_path):
    dimensions = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Look for tables in each page
            tables = page.extract_tables()
            for table in tables:
                # Process each table row into a dictionary
                headers = ["Dim", "Description", "Comment", "Tol (-)", "Tol (+)", "XS", "S", "M", "L", "XL"]
                for row in table:
                    # if len(row) == len(headers):  # Ensure the row matches headers
                    #     print("u")
                    row_data = dict(zip(headers, row))
                    dimensions.append(row_data)
    return dimensions


In [37]:
def store_in_mongo(data, db_name, collection_name):
    # Connect to MongoDB
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client[db_name]
    collection = db[collection_name]
    
    # Insert data
    collection.insert_many(data)
    print(f"Inserted {len(data)} records into {collection_name} collection in {db_name} database.")


In [38]:
def main():
    pdf_path = r"E:\git\Automated-Email-Parsing-and-Document-Generation\input_doc.pdf"  # Path to the input PDF document
    dimensions_data = extract_dimensions(pdf_path)
    print(dimensions_data)
    # MongoDB storage
    db_name = "technical_specs"
    collection_name = "dimensions"
    store_in_mongo(dimensions_data, db_name, collection_name)

if __name__ == "__main__":
    main()


[{'Dim': 'Dim', 'Description': 'Description', 'Comment': 'Comment', 'Tol (-)': 'Tol\n(-)', 'Tol (+)': 'Tol\n(+)', 'XS': 'XS', 'S': None, 'M': None, 'L': 'S', 'XL': None}, {'Dim': '1B1', 'Description': 'ChestWidth', 'Comment': '', 'Tol (-)': '-1.00', 'Tol (+)': '1.00', 'XS': 'Increm\nent', 'S': 'Sam\nple', 'M': 'Devia\ntion', 'L': 'Incremen\nt', 'XL': 'Sampl\ne'}, {'Dim': '1X1', 'Description': 'BottomHem Width', 'Comment': '', 'Tol (-)': '-1.00', 'Tol (+)': '1.00', 'XS': '46.00', 'S': '', 'M': '', 'L': '50.00', 'XL': ''}, {'Dim': '1F3', 'Description': 'AcrossShoulder', 'Comment': '', 'Tol (-)': '-1.00', 'Tol (+)': '1.00', 'XS': '36.00', 'S': '', 'M': '', 'L': '38.00', 'XL': ''}, {'Dim': '1F6', 'Description': 'ShoulderAngle', 'Comment': '', 'Tol (-)': '-0.50', 'Tol (+)': '0.50', 'XS': '2.50', 'S': '', 'M': '', 'L': '2.50', 'XL': ''}, {'Dim': '1G5', 'Description': 'SleeveLength (Short)', 'Comment': '', 'Tol (-)': '-1.00', 'Tol (+)': '1.00', 'XS': '37.00', 'S': '', 'M': '', 'L': '39.00', '

In [39]:
# Connect to MongoDB
from pymongo import MongoClient
import pandas as pd

client = MongoClient("mongodb://localhost:27017/")
db = client["technical_specs"]
collection = db["dimensions"]

# Fetch all records
records = list(collection.find())

# Extract unique column names dynamically from MongoDB records
columns = set()
for record in records:
    columns.update(record.keys())

columns = list(columns)
columns.remove("_id")  # Remove MongoDB's default '_id' field if present
columns.sort()  # Optional: Sort the columns for better readability

print("Extracted Columns:", columns)


Extracted Columns: ['Comment', 'Description', 'Dim', 'L', 'M', 'S', 'Tol (+)', 'Tol (-)', 'XL', 'XS']


In [40]:
# Prepare a DataFrame using the extracted columns
df = pd.DataFrame(records)

# Remove the MongoDB '_id' column if it exists
if "_id" in df.columns:
    df = df.drop(columns=["_id"])

# Display the first few rows to verify the structure
df.head()


,Dim,Description,Comment,Tol (-),Tol (+),XS,S,M,L,XL
0,Dim,Description,Comment,Tol\n(-),Tol\n(+),XS,None,None,S,None
1,1B1,ChestWidth,,-1.00,1.00,Increm\nent,Sam\nple,Devia\ntion,Incremen\nt,Sampl\ne
2,1X1,BottomHem Width,,-1.00,1.00,46.00,,,50.00,
3,1F3,AcrossShoulder,,-1.00,1.00,36.00,,,38.00,
4,1F6,ShoulderAngle,,-0.50,0.50,2.50,,,2.50,


In [41]:
# Save the data to an Excel file
output_file = "dynamic_output.xlsx"
df.to_excel(output_file, index=False)

print(f"Data has been successfully saved to {output_file}")


Data has been successfully saved to dynamic_output.xlsx


In [42]:
#dataCombining----------------------------------------------------------------------

In [43]:
import pandas as pd

# Load the original Excel file containing all data
input_file = "dynamic_output.xlsx"  # Replace with the actual file path
data = pd.read_excel(input_file)

# Display the first few rows to understand the structure
data.head()


,Dim,Description,Comment,Tol (-),Tol (+),XS,S,M,L,XL
0,Dim,Description,Comment,Tol\n(-),Tol\n(+),XS,NaN,NaN,S,NaN
1,1B1,ChestWidth,NaN,-1.00,1.00,Increm\nent,Sam\nple,Devia\ntion,Incremen\nt,Sampl\ne
2,1X1,BottomHem Width,NaN,-1.00,1.00,46.00,NaN,NaN,50.00,NaN
3,1F3,AcrossShoulder,NaN,-1.00,1.00,36.00,NaN,NaN,38.00,NaN
4,1F6,ShoulderAngle,NaN,-0.50,0.50,2.50,NaN,NaN,2.50,NaN


In [44]:
# Identify rows for the first and second tables
table1_start = data.index[data['Dim'] == "Displaying 1-9 results"].tolist()
table2_start = data.index[data['Dim'] == "Displaying 10-15 results"].tolist()

# Extract the two tables based on the identified rows
if table1_start and table2_start:
    table1 = data.iloc[:table2_start[0]].reset_index(drop=True)
    table2 = data.iloc[table2_start[0]:].reset_index(drop=True)

# Drop rows that contain display headings
table1 = table1[table1['Dim'] != "Displaying 1-9 results"]
table2 = table2[table2['Dim'] != "Displaying 10-15 results"]


In [45]:
# Identify rows for the first and second tables
table1_start = data.index[data['Dim'] == "Displaying 1-9 results"].tolist()
table2_start = data.index[data['Dim'] == "Displaying 10-15 results"].tolist()

# Extract the two tables based on the identified rows
if table1_start and table2_start:
    table1 = data.iloc[:table2_start[0]].reset_index(drop=True)
    table2 = data.iloc[table2_start[0]:].reset_index(drop=True)

# Drop rows that contain display headings
table1 = table1[table1['Dim'] != "Displaying 1-9 results"]
table2 = table2[table2['Dim'] != "Displaying 10-15 results"]


In [46]:
# Combine both tables into a single DataFrame
combined_table = pd.concat([table1, table2], ignore_index=True)


In [47]:
# Save the combined table into an Excel file
output_combined_file = "combined_table_output.xlsx"
combined_table.to_excel(output_combined_file, index=False)

print(f"Combined table successfully saved to {output_combined_file}")


Combined table successfully saved to combined_table_output.xlsx


In [48]:
#----------------------------------------------------------------------------------

In [50]:
# Define paths
input_path = "dynamic_output.xlsx"  # Replace with actual file path
data = pd.read_excel(input_path)

# Process Table 1
table1 = data[data['Dim'] != "Displaying 1-9 results"]
table1 = table1.reset_index(drop=True)

# Process Table 2
table2 = data[data['Dim'] != "Displaying 10-15 results"]
table2 = table2.reset_index(drop=True)
table1
table2

,Dim,Description,Comment,Tol (-),Tol (+),XS,S,M,L,XL
0,Dim,Description,Comment,Tol\n(-),Tol\n(+),XS,NaN,NaN,S,NaN
1,1B1,ChestWidth,NaN,-1.00,1.00,Increm\nent,Sam\nple,Devia\ntion,Incremen\nt,Sampl\ne
2,1X1,BottomHem Width,NaN,-1.00,1.00,46.00,NaN,NaN,50.00,NaN
3,1F3,AcrossShoulder,NaN,-1.00,1.00,36.00,NaN,NaN,38.00,NaN
4,1F6,ShoulderAngle,NaN,-0.50,0.50,2.50,NaN,NaN,2.50,NaN
5,1G5,SleeveLength (Short),NaN,-1.00,1.00,37.00,NaN,NaN,39.00,NaN
6,1H2,ArmholeDepth,NaN,-0.50,0.50,21.20,NaN,NaN,22.00,NaN
7,2A1,Upper ArmWidth,NaN,-0.50,0.50,17.00,NaN,NaN,18.00,NaN
8,2K1,SleeveHem Width,NaN,-0.50,0.50,15.00,NaN,NaN,16.00,NaN
9,3B1,NecklineWidth,NaN,-0.50,0.50,16.20,NaN,NaN,"17,00",NaN
